# Numerical Simulation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

## Initial Value Problem

An initial value problem (IVP) for a given differential equation:
\begin{equation}
\dot{x} = h(x(t), t),
\end{equation}
and initial condition $x(0)$ is to solve for the trajectory $x(t)$ that satisfies the differential equation starting from $x(0)$.

#### Example: Simple IVP

Consider the simple initial value problem with:
\begin{equation}
\dot{x} = x \sin^2(t), \quad x(0) = 1,
\end{equation}
which has an analytical solution:
\begin{equation}
    x(t) = x_0 \exp\left( \frac{t - t_0 - \sin(t-t_0)\cos(t+t_0)}{2} \right),
\end{equation}
which for the provided initial condition simplifies to:
\begin{equation}
x(t) = \exp\left( \frac{t - \sin(t) \cos(t)}{2} \right).
\end{equation}
Our goal is to develop a numerical integration scheme which can approximate this solution. We can define the IVP using the code below:

In [ ]:
# Define the derivative function h(x, t)
def h(x, t):
    return x * np.sin(t) ** 2

# Define initial conditions
x0 = np.array([1.0]) # Initial state x(t0)
t0 = 0.0 # Initial time t0
tf = 10.0 # Final time tf

## Numerical Integration Schemes

There are many methods for numerical integration, with varying degrees of accuracy and computational intensity. One of the simplest integration schemes is the Euler method, which uses the update formula:
\begin{equation}
x(t + \Delta t) = x(t) + \Delta t \cdot h(x(t), t),
\end{equation}
to integrate the ordinary differential equation:
\begin{equation}
\dot{x} = h(x(t), t).
\end{equation}

To implement such a numerical integration scheme to solve an initial value problem (IVP), we start with a simple for-loop that computes a sequence of single-step updates. See the function `integrate` defined below.

In [ ]:
def integrate(h, x0, t, method, **kwargs):
    """
    Perform numerical integration of ̇x = h(x,t) from initial state x0.

    Parameters
    ----------
    h : function to integrate, h(x, t, **kwargs)
    x0 : initial state vector
    t : time vector [t0, t0 + Δt, ..., T] 
    method : integration method function, integrator (h, x, t, Δt, **kwargs) 
    kwargs : key word args to pass to h

    Returns
    -------
    x : state vector for times t
    """
    x = np.zeros((t.size, x0.size))
    x[0] = x0
    for i in range(t.size - 1):
        Δt = t[i+1] - t[i]
        x[i + 1] = method(h, x[i], t[i], Δt, **kwargs)
    return x

We can now implement the Euler, midpoint, and fourth-order Runge-Kutta methods that we can pass to `integrate`. The midpoint method and the fourth-order Runge-Kutta (RK4) method are two alternatives to the Euler method with better accuracy.

In [ ]:
def euler(h, x, t, Δt, **kwargs):
    """
    Perform Euler integration step of ̇x = h(x,t)

    Parameters
    ----------
    h : function to integrate, h(x, t, **kwargs)
    x : state vector
    t : time
    Δt : time step to integrate over
    kwargs : key word args to pass to h

    Returns
    -------
    x[t + Δt] : state vector at t + Δt
    """
    return x + Δt*h(x, t, **kwargs)

In [ ]:
def midpoint(h, x, t, Δt, **kwargs):
    """
    Perform midpoint integration step of ̇x = h(x,t)

    Parameters
    ----------
    f : function to integrate, h(x, t, **kwargs)
    x : state vector
    t : time
    Δt : time step to integrate over
    kwargs : key word args to pass to h

    Returns
    -------
    x[t + Δt] : state vector at t + Δt
    """
    t_mid = t + Δt/2
    x_mid = x + (Δt/2)*h(x, t)
    return x + Δt*h(x_mid, t_mid)

In [ ]:
def rk4(h, x, t, Δt, **kwargs):
    """
    Perform fourth-order Runge-Kutta integration step of ̇x = h(x,t)

    Parameters
    ----------
    h : function to integrate, h(x, t, **kwargs)
    x : state vector
    t : time
    Δt : time step to integrate over
    kwargs : key word args to pass to h

    Returns
    -------
    x[t + Δt] : state vector at t + Δt
    """
    k1 = h(x, t)
    k2 = h(x + (Δt/2)*k1, t + Δt/2)
    k3 = h(x + (Δt/2)*k2, t + Δt/2)
    k4 = h(x + Δt*k3, t + Δt)
    return x + (Δt/6)*(k1 + 2*k2 + 2*k3 + k4)

### Exercise 1: Simple IVP Integration

For our first exercise, we will solve the IVP from the simple example above using the Euler, midpoint, and fourth-order Runge-Kutta methods and compare against the analytical solution to see the differences in their accuracy.

Run the code below to compare the numerical and analytical solutions to the IVP. Try playing with the $\Delta t$ parameter to see how the numerical method's accuracy varies.

In [ ]:
Δt = 0.5 # Discretization step
t = np.arange(t0, tf + Δt, Δt) # Array of timestamps

# Compute analytical solution
x_true = x0*np.exp(((t-t0) - np.sin(t-t0)*np.cos(t+t0))/2)

# Test Euler method
x_euler = integrate(h, x0, t, euler)

# Test midpoint method
x_midpoint = integrate(h, x0, t, midpoint)

# Test RK4 method
x_rk4 = integrate(h, x0, t, rk4)

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=150)
ax.plot(t, x_true, 'k', label='true')
ax.plot(t, x_euler, '--', label='euler')
ax.plot(t, x_midpoint, '--', label='midoint')
ax.plot(t, x_rk4, '--', label='rk4')
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$x$')
ax.legend()
plt.show()

# Robot Motion Models

In this section of the notebook we will further explore implementing dynamics models and simulating them using the methods above.

### Damped Pendulum

The dynamics for a simple (damped) pendulum is defined by the second order equation:
\begin{equation}
\ddot{\theta} = -\frac{g}{l}\sin \theta - \gamma \dot{\theta},
\end{equation}
where $\theta$ is the angle of the pendulum with respect to the negative vertical axis, $g$ is the acceleration due to gravity, $l$ is the length of the pendulum, and $\gamma$ is a damping coefficient.
This second-order differential equation is equivalent to the system of first-order differential equations:
\begin{equation}
\dot{x}_0 = x_1, \quad \dot{x}_1 = -\frac{g}{l}\sin x_0 - \gamma x_1,
\end{equation}
where $x_0 = \theta$ and $x_1 = \dot{\theta}$.

In [ ]:
def dampled_pendulum(x, t, g=9.81, l=1., γ=1.):
    """
    Evaluate pendulum dynamics.

    Parameters
    ----------
    x : state vector, [θ, dθ_dt]
    t : time [s]
    g : acceleration due to gravity [m/s^2]
    l : pendulum length [m]
    γ : damping coefficient [1/s]

    Returns
    -------
    dx_dt : state vector derivative, [dθ_dt, d2θ_dt2]
    """
    θ, dθ = x
    ddθ = -(g/l)*np.sin(θ) - γ*dθ
    dx = np.array([dθ, ddθ])
    return dx

We can simulate general robot motion models by solving the differential equations via numerical integration, for example using the methods discussed above. In Python, the `SciPy` package also offers a method for numerical integration called `odeint` that uses a more advanced method than those we experimented with so far. Run the code block below to simulated the damped pendulum.

In [ ]:
# Define IVP and model constants
θ0 = np.deg2rad(30.) # initial angle
x0 = np.array([θ0, 0.])
T = 10. # simulation time
l = 1 # pendulum length
γ = 0.5 # damping coefficient

# Use odeint to simulated the dynamics
t = np.linspace(0, T, num=100)
x_odeint = odeint(dampled_pendulum, x0, t, args=(9.81, 1, γ))
θ_odeint, dθ_odeint = x_odeint.T

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=150)
ax.plot(t, np.rad2deg(θ_odeint))
ax.set_title(r'$l = $' + f'{l:g}, ' + r'$\gamma = $' + f'{γ:g}')
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$\theta(t)$ [deg]')
plt.show()

### Exercise 2: Simulate Bicycle Dynamics

A simple motion model for a bicycle/simple car is:
\begin{equation}
\dot{x} = v\cos\theta, \quad \dot{y} = v\sin\theta, \quad \dot{\theta} = \frac{v}{L}\tan\phi, 
\end{equation}
where $(x,y)$ is the position, $v$ is the speed control, $\theta$ is the heading angle, $L$ is the wheelbase length, and $\phi$ is the steering angle.
This is a controlled system with the control inputs being $u = [v, \phi]^\top$.
Here we can assume simple simple open-loop control with:
\begin{equation}
v = 1, \quad \phi = \sin(t).
\end{equation}

Implement the `open_loop_bicycle` function to model the bicycle/simple car dynamics with the specified open-loop controller.

In [ ]:
def open_loop_bicycle(x, t, L=1.):
    """
    Evaluate bicycle dynamics with a simple open-loop control law.

    Parameters
    ----------
    x : state vector, [x, y, θ]
    t : time [s]
    L : wheelbase length [m]

    Returns
    -------
    dx_dt : state vector derivative, [̇dx_dt, dy_dt, dθ_dt]
    """
    # Open-loop controller definition
    v = 1 # constant speed
    ϕ = np.sin(t)

    ### Implement bicycle dynamics here
    dx_dt = np.zeros_like(x)
    return dx_dt

 Run the script below to simulate the dynamics using the RK4 integration method from above.

In [ ]:
Δt = 0.01
T = 10. # simulation time
t = np.linspace(0, T, num=int(T/Δt))

# Use RK4 method to simulate
x0 = np.array([0, 0, 0])
x_bike = integrate(open_loop_bicycle, x0, t, rk4, L=1)
pos_x_bike, pos_y_bike, θ_bike = x_bike.T

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150)
ax.plot(pos_x_bike, pos_y_bike)
ax.set_xlabel(r'$x(t)$')
ax.set_ylabel(r'$y(t)$')
plt.show()